<a href="https://colab.research.google.com/github/lilianli17/RSNA_fracture_detection/blob/main/RSNA_Segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install pydicom
# !pip install python-gdcm
!pip -q install monai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import os
import random
import pydicom
import numpy as np
import pandas as pd
import nibabel as nib
import glob
from glob import glob
import cv2
import matplotlib.pyplot as plt
from monai.transforms import Resize
from sklearn.model_selection import KFold, StratifiedKFold

import torch
from torch.utils.data import DataLoader, Dataset

Congfiguration

In [6]:
n_folds = 5
resize_shape = (192, 192, 192)

data_dir = '/content/drive/MyDrive/rsna-2022-cervical-spine-fracture-detection'


Dataframe

In [7]:
df_train = pd.read_csv(os.path.join(data_dir, 'train.csv'))
df_test = pd.read_csv(os.path.join(data_dir, 'test.csv'))
df_bounding_boxes = pd.read_csv(os.path.join(data_dir, 'train_bounding_boxes.csv'))
df_train

,StudyInstanceUID,patient_overall,C1,C2,C3,C4,C5,C6,C7
0,1.2.826.0.1.3680043.6200,1,1,1,0,0,0,0,0
1,1.2.826.0.1.3680043.27262,1,0,1,0,0,0,0,0
2,1.2.826.0.1.3680043.21561,1,0,1,0,0,0,0,0
3,1.2.826.0.1.3680043.12351,0,0,0,0,0,0,0,0
4,1.2.826.0.1.3680043.1363,1,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...
2014,1.2.826.0.1.3680043.21684,1,0,1,0,0,0,1,1
2015,1.2.826.0.1.3680043.4786,1,0,0,0,0,0,0,1
2016,1.2.826.0.1.3680043.14341,0,0,0,0,0,0,0,0
2017,1.2.826.0.1.3680043.12053,0,0,0,0,0,0,0,0


In [8]:
# returns a list containing the names of the entries in the directory
df_mask = pd.DataFrame({
    'mask_file': os.listdir(f'{data_dir}/segmentations'),
})
# get rid to ".nii"
df_mask['StudyInstanceUID'] = df_mask['mask_file'].apply(lambda x: x[:-4])
# get directories segmentation files
df_mask['mask_file'] = df_mask['mask_file'].apply(lambda x: os.path.join(data_dir, 'segmentations', x))
df_mask

,mask_file,StudyInstanceUID
0,/content/drive/MyDrive/rsna-2022-cervical-spin...,1.2.826.0.1.3680043.10633
1,/content/drive/MyDrive/rsna-2022-cervical-spin...,1.2.826.0.1.3680043.10921
2,/content/drive/MyDrive/rsna-2022-cervical-spin...,1.2.826.0.1.3680043.11827
3,/content/drive/MyDrive/rsna-2022-cervical-spin...,1.2.826.0.1.3680043.11988
4,/content/drive/MyDrive/rsna-2022-cervical-spin...,1.2.826.0.1.3680043.12281
...,...,...
82,/content/drive/MyDrive/rsna-2022-cervical-spin...,1.2.826.0.1.3680043.8330
83,/content/drive/MyDrive/rsna-2022-cervical-spin...,1.2.826.0.1.3680043.8574
84,/content/drive/MyDrive/rsna-2022-cervical-spin...,1.2.826.0.1.3680043.8884
85,/content/drive/MyDrive/rsna-2022-cervical-spin...,1.2.826.0.1.3680043.8744


In [9]:
# df_total: containing all files w/ mask
df_total = df_train.merge(df_mask, on='StudyInstanceUID', how='left')
df_total['image_folder'] = df_total['StudyInstanceUID'].apply(lambda x: os.path.join(data_dir, 'train_images', x))
df_total['mask_file'].fillna('', inplace=True)
df_total

,StudyInstanceUID,patient_overall,C1,C2,C3,C4,C5,C6,C7,mask_file,image_folder
0,1.2.826.0.1.3680043.6200,1,1,1,0,0,0,0,0,,/content/drive/MyDrive/rsna-2022-cervical-spin...
1,1.2.826.0.1.3680043.27262,1,0,1,0,0,0,0,0,,/content/drive/MyDrive/rsna-2022-cervical-spin...
2,1.2.826.0.1.3680043.21561,1,0,1,0,0,0,0,0,,/content/drive/MyDrive/rsna-2022-cervical-spin...
3,1.2.826.0.1.3680043.12351,0,0,0,0,0,0,0,0,,/content/drive/MyDrive/rsna-2022-cervical-spin...
4,1.2.826.0.1.3680043.1363,1,0,0,0,0,1,0,0,/content/drive/MyDrive/rsna-2022-cervical-spin...,/content/drive/MyDrive/rsna-2022-cervical-spin...
...,...,...,...,...,...,...,...,...,...,...,...
2014,1.2.826.0.1.3680043.21684,1,0,1,0,0,0,1,1,,/content/drive/MyDrive/rsna-2022-cervical-spin...
2015,1.2.826.0.1.3680043.4786,1,0,0,0,0,0,0,1,,/content/drive/MyDrive/rsna-2022-cervical-spin...
2016,1.2.826.0.1.3680043.14341,0,0,0,0,0,0,0,0,,/content/drive/MyDrive/rsna-2022-cervical-spin...
2017,1.2.826.0.1.3680043.12053,0,0,0,0,0,0,0,0,,/content/drive/MyDrive/rsna-2022-cervical-spin...


In [10]:
# return the ones with segmentations
df_seg = df_total.query('mask_file != ""').reset_index(drop=True)
df_seg

,StudyInstanceUID,patient_overall,C1,C2,C3,C4,C5,C6,C7,mask_file,image_folder
0,1.2.826.0.1.3680043.1363,1,0,0,0,0,1,0,0,/content/drive/MyDrive/rsna-2022-cervical-spin...,/content/drive/MyDrive/rsna-2022-cervical-spin...
1,1.2.826.0.1.3680043.25704,0,0,0,0,0,0,0,0,/content/drive/MyDrive/rsna-2022-cervical-spin...,/content/drive/MyDrive/rsna-2022-cervical-spin...
2,1.2.826.0.1.3680043.20647,0,0,0,0,0,0,0,0,/content/drive/MyDrive/rsna-2022-cervical-spin...,/content/drive/MyDrive/rsna-2022-cervical-spin...
3,1.2.826.0.1.3680043.31077,1,0,0,1,1,1,1,0,/content/drive/MyDrive/rsna-2022-cervical-spin...,/content/drive/MyDrive/rsna-2022-cervical-spin...
4,1.2.826.0.1.3680043.17960,0,0,0,0,0,0,0,0,/content/drive/MyDrive/rsna-2022-cervical-spin...,/content/drive/MyDrive/rsna-2022-cervical-spin...
...,...,...,...,...,...,...,...,...,...,...,...
82,1.2.826.0.1.3680043.32071,1,0,1,0,1,0,1,1,/content/drive/MyDrive/rsna-2022-cervical-spin...,/content/drive/MyDrive/rsna-2022-cervical-spin...
83,1.2.826.0.1.3680043.30524,1,0,0,0,0,0,1,1,/content/drive/MyDrive/rsna-2022-cervical-spin...,/content/drive/MyDrive/rsna-2022-cervical-spin...
84,1.2.826.0.1.3680043.28025,0,0,0,0,0,0,0,0,/content/drive/MyDrive/rsna-2022-cervical-spin...,/content/drive/MyDrive/rsna-2022-cervical-spin...
85,1.2.826.0.1.3680043.21321,1,1,1,1,0,0,0,1,/content/drive/MyDrive/rsna-2022-cervical-spin...,/content/drive/MyDrive/rsna-2022-cervical-spin...


In [11]:
# resize and normalize dicom files
imagePaths_train = df_seg['image_folder']
# size = 86
resized_dicom = []

# resizing and normalizing
for counter in range(len(imagePaths_train)):
    dcm_files = []
    
    # dcm in 31077 folder are compressed
    if int(imagePaths_train[counter].split('/')[-1].split(".")[-1]) != 31077:
        print(int(imagePaths_train[counter].split('/')[-1].split(".")[-1]))
        for file in sorted(glob(os.path.join(imagePaths_train[6], "*")), 
                           key=lambda x: int(x.split('/')[-1].split(".")[0])):
            if file not in dcm_files:
                dicom_arr = pydicom.dcmread(file).pixel_array
                # resizing
                dicom_arr = cv2.resize(dicom_arr, (resize_shape[0], resize_shape[1]), cv2.INTER_LANCZOS4)
                # normalizing
                dicom_arr = np.uint8((dicom_arr / np.max(dicom_arr)) * 255)
                dcm_files.append(dicom_arr)
        resized_dicom.append(dcm_files)


1363


<ipython-input-11-6b05d6416b83>:15: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if file not in dcm_files:


25704
20647
17960
1573
18935
1868
26442
27752
12281
26068
8024
11827
1542
26740
26898
16092
8744
8884
5002
3992
20120
780
30067
5671
18968
26979
21651
17481
27016
1480
18480
19333
2243
26110
6125
4202
3882
10633
6376
19021
6078
11988
24617
10921
28665
4769
25833
5782
20928
23904
3168
9926
5783
32280
32590
26492
30565
32434
16919
30487
24891
32370
30640
32658
24606
18906
3376
12292
15206
8330
24140
32436
28327
19388
27292
8574
26498
12833
14267
29425
32071
30524
28025
21321
26990


In [ ]:
maskPaths_train = df_seg['mask_file']
mask_file = []
for counter in range(len(maskPaths_train)):
    
    # dcm in 31077 folder are compressed
    if int(maskPaths_train[counter].split('/')[-1].split(".")[-2]) != 31077:
        print(int(maskPaths_train[counter].split('/')[-1].split(".")[-2]))
        mask_org = nib.load(maskPaths_train[counter]).get_fdata() # numpy.memmap type
        shape = mask_org.shape

        # why transpose and reverse
        mask_org = mask_org.transpose(1, 0, 2)[::-1, :, ::-1]  # (d, w, h)

        # mask: numpy arrays from 7 features respectively
        mask = np.zeros((7, shape[0], shape[1], shape[2]))
        for cid in range(7):
            mask[cid] = (mask_org == (cid+1))
        # normalization
        mask = mask.astype(np.uint8) * 255
        
        # resizing using monai resize
        R = Resize(resize_shape)
        mask = R(mask).numpy()

        mask_file.append(mask)



1363
25704
20647
17960
1573
18935
1868
26442
27752
12281
26068
8024
11827
1542
26740
26898
16092
8744
8884
5002
3992
20120
780
30067
5671
18968
26979
21651
17481
27016
1480
18480
19333
2243
26110
6125
4202
3882
10633
6376
19021
6078
11988
24617
10921
28665
4769
25833
5782
20928
23904
3168
9926
5783
32280
32590
26492
30565
32434
16919
30487
24891
32370
30640


In [ ]:
# k folds
kf = KFold(5)
df_seg['fold'] = -1
# labeling df_seg in each fold
a = 2
for fold, (train_idx, valid_idx) in enumerate(kf.split(df_seg, df_seg)):
    df_seg.loc[valid_idx, 'fold'] = fold
df_seg

,StudyInstanceUID,patient_overall,C1,C2,C3,C4,C5,C6,C7,mask_file,image_folder,fold
0,1.2.826.0.1.3680043.1363,1,0,0,0,0,1,0,0,/content/drive/MyDrive/rsna-2022-cervical-spin...,/content/drive/MyDrive/rsna-2022-cervical-spin...,0
1,1.2.826.0.1.3680043.25704,0,0,0,0,0,0,0,0,/content/drive/MyDrive/rsna-2022-cervical-spin...,/content/drive/MyDrive/rsna-2022-cervical-spin...,0
2,1.2.826.0.1.3680043.20647,0,0,0,0,0,0,0,0,/content/drive/MyDrive/rsna-2022-cervical-spin...,/content/drive/MyDrive/rsna-2022-cervical-spin...,0
3,1.2.826.0.1.3680043.31077,1,0,0,1,1,1,1,0,/content/drive/MyDrive/rsna-2022-cervical-spin...,/content/drive/MyDrive/rsna-2022-cervical-spin...,0
4,1.2.826.0.1.3680043.17960,0,0,0,0,0,0,0,0,/content/drive/MyDrive/rsna-2022-cervical-spin...,/content/drive/MyDrive/rsna-2022-cervical-spin...,0
...,...,...,...,...,...,...,...,...,...,...,...,...
82,1.2.826.0.1.3680043.32071,1,0,1,0,1,0,1,1,/content/drive/MyDrive/rsna-2022-cervical-spin...,/content/drive/MyDrive/rsna-2022-cervical-spin...,4
83,1.2.826.0.1.3680043.30524,1,0,0,0,0,0,1,1,/content/drive/MyDrive/rsna-2022-cervical-spin...,/content/drive/MyDrive/rsna-2022-cervical-spin...,4
84,1.2.826.0.1.3680043.28025,0,0,0,0,0,0,0,0,/content/drive/MyDrive/rsna-2022-cervical-spin...,/content/drive/MyDrive/rsna-2022-cervical-spin...,4
85,1.2.826.0.1.3680043.21321,1,1,1,1,0,0,0,1,/content/drive/MyDrive/rsna-2022-cervical-spin...,/content/drive/MyDrive/rsna-2022-cervical-spin...,4
